<h1>Basics of Mobile Robotics project<span class="tocSkip"></span></h1>

**Group 23 : Diana Bejan, Emilie Grandjean, Garance Boesinger, Juan Martín**

<h1>Table of Contents<span class="tocSkip"></span></h1>

<div class="toc"><ul class="toc-item">
    <li><span><a href="#Introduction" data-toc-modified-id=">Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item">
        <li><span><a href="#Project-description" data-toc-modified-id="Project-description-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Project guidelines</a></span></li>
        <li><span><a href="#General-implementation-description" data-toc-modified-id="General-implementation-description-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>General implementation description</a></span></li>
        <li><span><a href="#Environment-Setup" data-toc-modified-id="Environment-Setup-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Environment Setup</a></span></li>
        <li><span><a href="#Libraries-Import" data-toc-modified-id="Libraires-Import-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Libraries Import</a></span></li>
        <li><span><a href="#Thymio COnnection" data-toc-modified-id="Thymio-Connection-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Thymio Connection</a></span></li>
        </ul></li>
    <li><span><a href="#Description-of-the-modules" data-toc-modified-id="Description-of-the-modules-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Description of the Modules</a></span><ul class="toc-item">
        <li><span><a href="#Vision" data-toc-modified-id="Vision-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Vision</a></span></li>
        <li><span><a href="#Global-Navigation" data-toc-modified-id="Global-Navigation-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Global Navigation</a></span></li>
        <li><span><a href="#Local-Navigation" data-toc-modified-id="Local-Navigation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Local Navigation</a></span></li>
        <li><span><a href="#Filtering" data-toc-modified-id="Filtering-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Filtering</a></span></li>
        <li><span><a href="#Motion-Control" data-toc-modified-id="Motion-Control-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Motion Control</a></span></li>
        </ul></li>
     <li><span><a href="#Bonus" data-toc-modified-id="Bonus-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Bonus</a></span></li>
    <li><span><a href="#Code-of-the-overall-project" data-toc-modified-id="Code-of-the-overall-project-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Code of the overall project</a></span></li>
    <li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Conclusion</a></span>
    </li></ul>
    
</div>

# Introduction

## Project description

1. **Create an environnment :** Our environment has to contain a set of obstacles that the Thymio avoids through *global navigation*. That is to say, the Thymio should avoid the obstacles without using the sensors to detect them.

2. **Find the best path :** The obective is that the Thymio goes from an *arbitrary point* in the map to a *target* that can be placed <u>anywhere</u> in the environment.  These will be changed during the demo to see how the system performs.

3. **Motion Control & Position estimation:** We will *control* the robot to help it move along the path. This requires an accurate estimate of the position of the robot which we will have to obtain through *bayesian filtering*.
   
4. **Avoid Obstacles :** While navigating, the Thymio will have to use *local navigation* to avoid *physical obstacals* that can be put in its path <u>at any point in time</u>. 



## Environment Setup  GARANCE

environment : the static obstacles have to be in 2D (it will be easier to detect with the camera than if it’s in 3D because the shadows are hard to detect and so it’s better to do it that way (heard an assistant say it))

quick explication on what our setup looks like, the use of the colors with pitcures


## Best Path Calculations KIKE

short explication on what method used and why and how and quickly mention that it uses the info send by the CV picked up by the environment

## Motion Control DIANA

moving the Thymio along the given trajectory while knowing the thymio’s position (use of a proportional controller with the angle between the thymio’s orientation and the destination point)

small descritption of the method used and why and what elements from the path needed


## Obstacles avoidance EMILIE

while the Thymio navigates, use of local navigation to avoid physical (3D) obstacles that can be put in its path at anytime (create hollow obstacles so they are not detected by the camera for the global navigation in the environment) —> use the exercise sessions on local navigation, adapt it to our case

explication on what the obstacles look like, short description sans plus

# Required Components 

## Computer Vision GARANCE

use of the camera (placed above the environment) —> locate the environment (edges etc.), the static (2D) obstacles and the thymio
	—> only do image processing (quickly saw in an exercise session) and use the given file explaining what to do, how to extract and process the image data.
	—> use AR tags : the camera can practically do everything on it’s own
	—> ! the camera does not use the same parameters for macs and windows => the one doing this will be the only one


details explications : step by step, accompagné by images and pieces of code of

on how it's done, what  calculations used, what variables used, the form of the vision file(is it a class, just a fct ? why ?)

++ add images to go with the description to illustrate the explications

detail explications on the differents functions called and what their inputs are and what their outputs are (can be done in un tableau for example), explain their use and the goal of using them and when they are used (like it what function are they called etc )


separation in several parts ? the computation of the position of thymio? the environment ? the obstacles ? the goal ?

++ sources


## Global Navigation KIKE

use the location of the static obstacles given by the vision step and find the optimal path the thymio has to follow to reach the goal (no movement, just find the best trajectory)
	—> use of existing libraries !

  
details explications : step by step, accompagné by images(?)--> video showing the path followed ? and pieces of code of

on how it's done, what  calculations used, what variables used, the form of the vision file(is it a class, just a fct ? why ?)

++ add images or videos to go with the description to illustrate the explications

detail explications on the differents functions called and what their inputs are and what their outputs are (can be done in un tableau for example), explain their use and the goal of using them and when they are used (like it what function are they called etc )


separation in several parts ? the computation of the position of thymio? the environment ? the obstacles ? the goal ? how was the algo written and used ? what does it send to who (MC) 
--> the implementation, the algorithm, ...

is it optimal ? yes ok, no, why ? 

++ sources


## Motion Control DIANA

use the trajectory given in the global navigation step to direct and move the thymio to the goal destination
	—> use of a P controller

details explications : step by step, accompagné by images(?)--> video showing the path followed ? and pieces of code of

on how it's done, what  calculations used, what variables used, the form of the vision file(is it a class, just a fct ? why ?)

++ add images or videos to go with the description to illustrate the explications

detail explications on the differents functions called and what their inputs are and what their outputs are (can be done in un tableau for example), explain their use and the goal of using them and when they are used (like it what function are they called etc )


separation in several parts ? the computation of the position of thymio? the environment ? the obstacles ? the goal ? how was the algo written and used ? what does it send to who (MC) 
--> the implementation, the algorithm, ...

is it optimal ? yes ok, no, why ? 

++ sources


## Local Navigation
  

## Filtering

# VIDEOS ? IMAGES ? LITTLE EXTRAS ?

# Overall Project

**do we have to put the main here ? or a code that called our main.py ? to be asked on tuesday maybe**

# Conclusion

**we need to write a conclusion**